In [21]:
import pickle
import gzip
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from PIL import Image
import os
np.random.seed(666)

In [103]:
def softmax(weights,train_data):
    bias = np.ones((np.shape(train_data)[0],1))
    train_withBias = np.hstack((train_data,bias))
    num = np.dot(weights,train_withBias.T)
    # High value Fix
    # https://houxianxu.github.io/2015/04/23/logistic-softmax-regression/
    num = np.subtract(num,np.max(num,axis=0))
    num = np.exp(num)
    # Fix softmax when using batch size 1 the dimension of deno changes
    if(len(train_data) == 1):
        deno = np.sum(num,axis=0)
    else:
        deno = np.sum(num,axis=1)
        deno = deno.reshape((10,1))
    return np.divide(num,deno)

def accuracy(predicted,target):
    correct = 0
    confusion_mat = np.zeros((10,10))
    for i in range(len(target)):
        if(predicted[i] == target[i]):
            correct+=1
        confusion_mat[target[i]][predicted[i]] =confusion_mat[target[i]][predicted[i]] +1
    return correct/len(target),pd.DataFrame(np.matrix(confusion_mat,dtype="int32"))

def one_hot_vect(tuple_data,classes):
    one_hot_encoded=np.zeros((len(tuple_data[1]),len(classes)))
    identity = np.identity(len(classes))
    for i in range(len(tuple_data[1])):
        one_hot_encoded[i] = np.add(one_hot_encoded[i],identity[tuple_data[1][i]])
    return one_hot_encoded

def more_metrics(conf_mat):
    true_positives = 0
    precision = []
    recall = []
    for i in range(len(conf_mat)):
        true_positives += conf_mat.iloc[i,i]
    conf_mat = np.matrix(conf_mat)
    tp_fp = np.array(np.sum(conf_mat,axis=1)).ravel()
    relevant_elements = np.array(np.sum(conf_mat,axis=0)).ravel()
    for i in range(len(conf_mat)):
        precision.append(conf_mat[i,i]/tp_fp[i])
        recall.append(conf_mat[i,i]/relevant_elements[i])
    return true_positives,precision,recall

# Importing Models

In [3]:
#np.savetxt('./models/logisticModel', weights)
#np.loadtxt('./models/logisticModel')

In [62]:
filename = '../mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()
train_data = np.append(training_data[0],validation_data[0],axis=0)
train_target = np.append(training_data[1],validation_data[1])
test_target = test_data[1]
test_data = test_data[0]

In [63]:
scaler = StandardScaler()
scaler.fit(train_data)
processed_train_data = scaler.transform(train_data)
scaler.fit(test_data)
processed_test_data = scaler.transform(test_data)

In [5]:
#rf = joblib.load("./models/randomForestModel.joblib")
#lr = np.loadtxt("./models/logisticModel")
#nn = joblib.load("./models/DNN_lowHidden.joblib")
#svm model pending

Using TensorFlow backend.


In [15]:
# Preserve order since LR is hardcoded and methods like rf depend on predic_proba :3
classifiers = [lr,nn,rf]

In [13]:
def ensemble(classifiers,processed_test_data,test_data,weights):
    num_classifiers = len(classifiers)
    print("Working on Logistic Regression")
    lr_pred = np.multiply(softmax(classifiers[0],processed_test_data),weights[0])
    print("Working on Neural Network")
    nn_pred = np.multiply(classifiers[1].predict(test_data,verbose=True),weights[1])
    print("Working on Random Forest")
    rf_pred = np.multiply(classifiers[2].predict_proba(processed_test_data),weights[2])
    sumProb = (np.transpose(lr_pred)+rf_pred+nn_pred)
    wtAvg = np.divide(sumProb,len(classifiers))
    return np.argmax(wtAvg,axis=1)

In [69]:
# Brute Force to find best weights
def findWeights(classifiers,processed_test_data,test_data,test_target,weights)
    tracker = []
    for weight1 in range(1,4):
        for weight2 in range(1,4):
            for weight3 in range(1,4):
                if(not(weight1 == weight2 == weight3)):
                    weights=[weight1,weight2,weight3]
                    predicted = ensemble(classifiers,processed_test_data,test_data,weights)
                    acc,_ = accuracy(predicted,test_target)
                    print()
                    print("For weights "+str(weights)+" the accuracy is: "+str(acc))
                    tracker.append([weights,acc])
                    print()
    max_acc = 0
    tracker = np.asarray(tracker)
    for i,sample in enumerate(tracker):
    if(max_acc<tracker[i][1]):
        max_acc=tracker[i][1]
        opt_weight = tracker[1][0]
    return opt_weight

Working on Logistic Regression
Working on Neural Network
60000/60000 [==============================] - 5s 77us/step
Working on Random Forest

For weights [1, 1, 2] the accuracy is: 0.1007

Working on Logistic Regression
Working on Neural Network
60000/60000 [==============================] - 5s 77us/step
Working on Random Forest

For weights [1, 1, 3] the accuracy is: 0.1008

Working on Logistic Regression
Working on Neural Network
60000/60000 [==============================] - 5s 78us/step
Working on Random Forest

For weights [1, 2, 1] the accuracy is: 0.1006

Working on Logistic Regression
Working on Neural Network
60000/60000 [==============================] - 5s 77us/step
Working on Random Forest

For weights [1, 2, 2] the accuracy is: 0.1006

Working on Logistic Regression
Working on Neural Network
60000/60000 [==============================] - 5s 77us/step
Working on Random Forest

For weights [1, 2, 3] the accuracy is: 0.1006

Working on Logistic Regression
Working on Neural N

In [89]:
opt_weight = findWeights(classifiers,processed_test_data,test_data,test_target,weights)

[1, 1, 3]

In [104]:
predicted = ensemble(classifiers,processed_test_data,test_data,opt_weight)
acc,conf_mat = accuracy(predicted,test_target)
print("The Accuracy for MNIST is: "+str(acc))
print("The Confusion Matrix is: ")
print(pd.DataFrame(conf_mat))
_,precision,recall = more_metrics(pd.DataFrame(conf_mat))
print("The Precision & Recall is: ")
df = pd.DataFrame(np.multiply(precision,100))
df.columns = ["Precision"]
df1 = pd.DataFrame(np.multiply(recall,100))
df1.columns = ["Recall"]
print(pd.concat([df,df1],axis=1))

Working on Logistic Regression
Working on Neural Network
10000/10000 [==============================] - 1s 80us/step
Working on Random Forest
The Accuracy for MNIST is: 0.9832
The Confusion Matrix is: 
     0     1     2    3    4    5    6     7    8    9
0  972     1     0    1    0    0    2     1    3    0
1    0  1129     1    2    0    1    0     2    0    0
2    2     0  1011    1    1    1    1    11    3    1
3    1     0     1  981    0   13    0     8    3    3
4    0     0     2    1  962    0    3     2    1   11
5    2     0     0    3    0  881    2     1    1    2
6    4     2     0    1    2    2  943     0    4    0
7    1     1     6    2    0    0    0  1016    2    0
8    0     0     2    4    3    2    1     5  953    4
9    1     1     0    2    7    5    1     5    3  984
The Precision & Recall is: 
   Precision     Recall
0  99.183673  98.880977
1  99.471366  99.559083
2  97.965116  98.826979
3  97.128713  98.296593
4  97.963340  98.666667
5  98.766816  97.3480

In [41]:
USPSMat  = []
USPSTar  = []
curPath  = '../USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

In [28]:
#target_USPS = one_hot_vect((1,USPSTar),np.unique(USPSTar))
scaler.fit(USPSMat)
processed_USPSDat = scaler.transform(USPSMat)


In [98]:
predicted = ensemble(classifiers,processed_USPSDat,[USPSMat],opt_weight)
acc,conf_mat = accuracy(predicted,USPSTar)
print("The Accuracy for USPS is: "+str(acc))
print("The Confusion Matrix is: ")
print(pd.DataFrame(conf_mat))
_,precision,recall = more_metrics(pd.DataFrame(conf_mat))
print("The Precision & Recall is: ")
df = pd.DataFrame(np.multiply(precision,100))
df.columns = ["Precision"]
df1 = pd.DataFrame(np.multiply(recall,100))
df1.columns = ["Recall"]
print(pd.concat([df,df1],axis=1))

Working on Logistic Regression
Working on Neural Network
19999/19999 [==============================] - 1s 73us/step
Working on Random Forest
The Accuracy for USPS is: 0.5059252962648132
The Confusion Matrix is: 
     0    1     2     3     4     5     6    7    8    9
0  836   71    54    13    32    54   116   48  150   37
1    4  575     4     4    71     3    15  234   12  104
2   66  144  1548    71    30    58   201  215   90   54
3   93  144    86  1465    23    68    44  566  328  259
4  186  321    13     3  1002     5    38   31   69  141
5  154  131    86   347   129  1647   226   50  362   93
6  112   91    79     6    34    15  1201    6   51    6
7  150  349    57    21   320    55    40  690   87  659
8   90   92    66    61   301    82    69  148  824  317
9  309   82     6     9    58    13    50   12   27  330
The Precision & Recall is: 
   Precision     Recall
0  59.248760  41.800000
1  56.042885  28.750000
2  62.494954  77.438719
3  47.626788  73.250000
4  55.389718